<a href="https://colab.research.google.com/github/gabrielvieiraf/ProjetosPython/blob/master/GoogleColab/MachineLearning/Machine_Learning04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

># Iniciando com Machine Learning 04
>### Introdução a sistemas de recomendações

> ## Importando dados do Google Drive
Utilizaremos a base de dados disponível para download em: 


https://grouplens.org/datasets/movielens/latest/

In [1]:
# Permite Importar dados do Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


> ## Iniciando visualização com Pandas
> Em primeiro momento, vamos importar os filmes

In [2]:
import pandas as pd

# Caminho para dados do arquivo csv 
csv = '/content/drive/My Drive/Colab Notebooks/Alura/filme_100mil/movies.csv'
filmes = pd.read_csv(csv)
filmes.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


> ### Traduzindo Colunas

Como queremos trazer maior facilidade de visualização de dados, vamos traduzir as colunas de nosso dataset, de uma maneira bem simples, utilizando o próprio pandas

In [3]:
filmes.columns = ['id_filme','titulo','genero']

# Definindo id como index do dataset
filmes = filmes.set_index("id_filme")
filmes.head()

,titulo,genero
id_filme,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,Jumanji (1995),Adventure|Children|Fantasy
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride Part II (1995),Comedy


> ## Prosseguindo com o Pandas
> Agora iremos importar também as notas


In [4]:
# Caminho para dados do arquivo csv 
csv = '/content/drive/My Drive/Colab Notebooks/Alura/filme_100mil/ratings.csv'
notas = pd.read_csv(csv)
notas.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


> ### Traduzindo Colunas

Do mesmo modo como fizemos com os filmes, vamos traduzir as colunas das notas

In [5]:
notas.columns = ['id_usuario','id_filme','nota','data']
notas.head()

,id_usuario,id_filme,nota,data
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


> ## Primeira Tentativa de Recomendação
>Verificando a popularidade de filmes para um usuário com dados desconhecidos

In [6]:
# Verificando número notas por filme id
total_votos = notas['id_filme'].value_counts()
total_votos

356       329
318       317
296       307
593       279
2571      278
         ... 
5986        1
100304      1
34800       1
83976       1
8196        1
Name: id_filme, Length: 9724, dtype: int64

> ### Verificando Número de Votos por Filme

Isso nos ajudará a conhecer os filmes mais famosos para recomendar ao usuário

In [7]:
filmes['total_votos'] = total_votos
filmes.head()

,titulo,genero,total_votos
id_filme,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0
2,Jumanji (1995),Adventure|Children|Fantasy,110.0
3,Grumpier Old Men (1995),Comedy|Romance,52.0
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0
5,Father of the Bride Part II (1995),Comedy,49.0


In [8]:
# Ordenando valores
filmes.sort_values('total_votos',ascending = False).head()

,titulo,genero,total_votos
id_filme,,,
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0
593,"Silence of the Lambs, The (1991)",Crime|Horror|Thriller,279.0
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0


> ### Verificando a média das notas dos filmes

In [9]:
media_notas = notas.groupby("id_filme").mean()['nota']

> ### Criando coluna para notas em filmes

In [10]:
filmes['media_nota'] = media_notas
filmes.head()

,titulo,genero,total_votos,media_nota
id_filme,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
2,Jumanji (1995),Adventure|Children|Fantasy,110.0,3.431818
3,Grumpier Old Men (1995),Comedy|Romance,52.0,3.259615
4,Waiting to Exhale (1995),Comedy|Drama|Romance,7.0,2.357143
5,Father of the Bride Part II (1995),Comedy,49.0,3.071429


>### Ordenando por filmes por notas 

Verificando problema de quantidade de numero total de votos

In [11]:
# Ordenando valores
filmes.sort_values('media_nota',ascending = False).head()

,titulo,genero,total_votos,media_nota
id_filme,,,,
88448,Paper Birds (Pájaros de papel) (2010),Comedy|Drama,1.0,5.0
100556,"Act of Killing, The (2012)",Documentary,1.0,5.0
143031,Jump In! (2007),Comedy|Drama|Romance,1.0,5.0
143511,Human (2015),Documentary,1.0,5.0
143559,L.A. Slasher (2015),Comedy|Crime|Fantasy,1.0,5.0


> #### Filtrando numero de notas por Query


In [12]:
filmes_200 = filmes.query('total_votos >= 200').sort_values('media_nota',ascending = False).head(10)
filmes_200

,titulo,genero,total_votos,media_nota
id_filme,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,204.0,4.237745
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,251.0,4.231076
527,Schindler's List (1993),Drama|War,220.0,4.225000
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Sci-Fi,211.0,4.215640
1198,Raiders of the Lost Ark (Indiana Jones and the...,Action|Adventure,200.0,4.207500
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
2571,"Matrix, The (1999)",Action|Sci-Fi|Thriller,278.0,4.192446


> ## Segunda Tentativa de Recomendação
> Recomendação baseada em similaridade de genero do filme


In [13]:
filmes_assistidos = [1,20,156,42,6]
filmes.loc[filmes_assistidos]

,titulo,genero,total_votos,media_nota
id_filme,,,,
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,215.0,3.920930
20,Money Train (1995),Action|Comedy|Crime|Drama|Thriller,15.0,2.500000
156,Blue in the Face (1995),Comedy|Drama,3.0,3.333333
42,Dead Presidents (1995),Action|Crime|Drama,7.0,3.000000
6,Heat (1995),Action|Crime|Thriller,102.0,3.946078


In [14]:
# Filtramos nossos filmes acima de 200 avaliações pelos gêneros comédia e drama
filmes_comedia_drama = filmes_200.query('genero.str.contains("Comedy|Drama")', engine='python')

# Retira filmes já assistidos, ordena pela maior nota os filmes de comédia ou/e drama
filmes_comedia_drama = filmes_comedia_drama.drop(filmes_assistidos,errors='ignore').sort_values('media_nota', ascending = False)
filmes_comedia_drama

,titulo,genero,total_votos,media_nota
id_filme,,,,
318,"Shawshank Redemption, The (1994)",Crime|Drama,317.0,4.429022
2959,Fight Club (1999),Action|Crime|Drama|Thriller,218.0,4.272936
527,Schindler's List (1993),Drama|War,220.0,4.225000
296,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,307.0,4.197068
356,Forrest Gump (1994),Comedy|Drama|Romance|War,329.0,4.164134


> ## Terceira tentativa de Recomendação
Procurando usuários similares

> ### Calculando a distância entre os pontos

In [15]:
import numpy as np

Gabriel_Vieira = np.array([4,5,2,0])
Carla = np.array([0,2,5,4])
Paulo = np.array([5,4,2,0])

def distancia(a,b):
  return np.linalg.norm(a -b)

print('Distância entre Gabriel e Carla:')
print('%.2f' % distancia(Gabriel_Vieira,Carla))
print()
print('Distância entre Gabriel e Paulo:')
print('%.2f' % distancia(Gabriel_Vieira,Paulo))

Distância entre Gabriel e Carla:
7.07

Distância entre Gabriel e Paulo:
1.41


> ### Implementando distância de pontos com os usuários

In [16]:
# Notas do usuario com id 1
notas_usuario = notas.query('id_usuario == 1')

# Dataframe com nota e id do filme como index
notas_usuario = notas_usuario[['id_filme','nota']].set_index('id_filme')

# Localizando filme de id 47
notas_usuario.loc[47]

nota    5.0
Name: 47, dtype: float64

> ### Generalizando em uma função notas_usuario



In [17]:
def notas_usuario(usuario):
  # Notas do usuario
  notas_usuario = notas.query('id_usuario == %d' % usuario)

  # Dataframe com nota e id do filme como index
  notas_usuario = notas_usuario[['id_filme','nota']].set_index('id_filme')

  return notas_usuario

usuario_1 = notas_usuario(1)
usuario_4 = notas_usuario(4)

# juntando dois dataframes e filtrando notas que apenas um dos usuarios submeteu
usuarios = usuario_1.join(usuario_4, lsuffix = '_user01', rsuffix = '_user04' ).dropna()

display(usuarios.head())

# Calculando distancia entre o usuario 01 e usuario 04
print('\nA distância entre os usuários 01 e 04 é de:')
print('%.2f' % distancia(usuarios['nota_user01'],usuarios['nota_user04']))


,nota_user01,nota_user04
id_filme,,
47,5.0,2.0
235,4.0,2.0
260,5.0,5.0
296,3.0,1.0
441,4.0,1.0



A distância entre os usuários 01 e 04 é de:
11.14


> ### Generalizando em uma função distância entre dois usuarios

In [18]:
def distancia_usuarios(id_01,id_02):
  usuario_1 = notas_usuario(id_01)
  usuario_2 = notas_usuario(id_02)

  # juntando dois dataframes e filtrando notas que apenas um dos usuarios submeteu
  usuarios = usuario_1.join(usuario_2, lsuffix = '_user%d' % id_01, rsuffix = '_user%d' % id_02 ).dropna()
  distancia_usuarios = distancia(usuarios['nota_user%d' % id_01],usuarios['nota_user%d' % id_02])
  return [id_01,id_02,distancia_usuarios]

# Calculando distancia entre o usuario 01 e usuario 04
print('Lista com retorno id 01, id 02 e distância:')
print(distancia_usuarios(1,4))

Lista com retorno id 01, id 02 e distância:
[1, 4, 11.135528725660043]


> ### Verificando quantidade de usuários

In [19]:
quantidade_usuarios = len(notas['id_usuario'].unique())
print('A quantidade de usuários é de:', quantidade_usuarios)

A quantidade de usuários é de: 610


> ### Calculando a distância entre um e todos os usuários


In [20]:
gabriel_id = 1

# Maneira imperativa 
def distancia_todos(ref_id):
  distancias = list()
  for usuario_id in notas['id_usuario'].unique():
    infos = distancia_usuarios(ref_id,usuario_id)
    distancias.append(infos)
  distancias = pd.DataFrame(distancias, columns = ['ref_id','outro_usuario_id','distancia'])
  return distancias

def distancia_todos(ref_id):
  todos_users = notas['id_usuario'].unique()
  distancias = [ distancia_usuarios(ref_id,usuario_id) for usuario_id in todos_users]
  distancias = pd.DataFrame(distancias, columns = ['ref_id','outro_usuario_id','distancia'])
  return distancias

distancia_todos(gabriel_id)

,ref_id,outro_usuario_id,distancia
0,1,1,0.000000
1,1,2,1.414214
2,1,3,8.200610
3,1,4,11.135529
4,1,5,3.741657
...,...,...,...
605,1,606,11.510864
606,1,607,9.899495
607,1,608,18.241436
608,1,609,3.162278


> ### Buscando usuários próximos

In [21]:
dt = distancia_todos(gabriel_id)
df_distancias = dt.query('outro_usuario_id != 1')
df_distancias.sort_values('distancia').head(10)

,ref_id,outro_usuario_id,distancia
305,1,306,0.0
577,1,578,0.0
555,1,556,0.0
76,1,77,0.0
84,1,85,0.0
174,1,175,0.0
252,1,253,0.0
290,1,291,0.0
505,1,506,0.0
495,1,496,0.0


> ### Arrumando problema de pessoas que não deram notas em filmes

In [22]:
def distancia_usuarios(id_01,id_02, minimo = 6):
  usuario_1 = notas_usuario(id_01)
  usuario_2 = notas_usuario(id_02)

  # juntando dois dataframes e filtrando notas que apenas um dos usuarios submeteu
  usuarios = usuario_1.join(usuario_2, lsuffix = '_user%d' % id_01, rsuffix = '_user%d' % id_02 ).dropna()

  if (len(usuarios) < minimo):
    return [id_01,id_02,100000]

  distancia_usuarios = distancia(usuarios['nota_user%d' % id_01],usuarios['nota_user%d' % id_02])
  return [id_01,id_02,distancia_usuarios]

# Maneira imperativa 
def distancia_todos(ref_id):
  distancias = list()
  for usuario_id in notas['id_usuario'].unique():
    infos = distancia_usuarios(ref_id,usuario_id)
    distancias.append(infos)
  distancias = pd.DataFrame(distancias, columns = ['ref_id','outro_usuario_id','distancia'])
  return distancias

def proximos(ref_id):
  distancia = distancia_todos(ref_id)
  return distancia.sort_values('distancia').query('outro_usuario_id != %d' % ref_id)


In [23]:
 proximos(1).head()

,ref_id,outro_usuario_id,distancia
76,1,77,0.000000
510,1,511,0.500000
522,1,523,1.000000
48,1,49,1.000000
397,1,398,1.224745


> ### Parâmetros para teste

In [24]:
def distancia_usuarios(id_01,id_02, minimo = 6):
  usuario_1 = notas_usuario(id_01)
  usuario_2 = notas_usuario(id_02)

  # juntando dois dataframes e filtrando notas que apenas um dos usuarios submeteu
  usuarios = usuario_1.join(usuario_2, lsuffix = '_user%d' % id_01, rsuffix = '_user%d' % id_02 ).dropna()

  if (len(usuarios) < minimo):
    return None

  distancia_usuarios = distancia(usuarios['nota_user%d' % id_01],usuarios['nota_user%d' % id_02])
  return [id_01,id_02,distancia_usuarios]


def distancia_todos(ref_id, n = None):
  todos_users = notas['id_usuario'].unique()
  if n:
    todos_users = todos_users[:n]
  distancias = [ distancia_usuarios(ref_id,usuario_id) for usuario_id in todos_users]
  # Filtrando
  distancias = list(filter(None,distancias))
  distancias = pd.DataFrame(distancias, columns = ['ref_id','outro_usuario_id','distancia'])
  return distancias


def proximos(ref_id, n = None):
  distancia = distancia_todos(ref_id, n = n)
  return distancia.sort_values('distancia').query('outro_usuario_id != %d' % ref_id)

In [25]:
proximos(1, n = 48).head()

,ref_id,outro_usuario_id,distancia
21,1,25,1.414214
9,1,13,1.414214
26,1,30,1.802776
31,1,35,2.236068
22,1,26,2.236068


> ### Buscando dados sobre usuário mais similar dentro de um conjunto de 48

In [26]:
similares = proximos(1, n = 48)
similar = similares.iloc[0].name

> ### Eliminando filmes já vistos e criando recomendações em ordem crescente dos filmes com maior nota do usuário semelhante


In [27]:
notas_similar = notas_usuario(similar)
filmes_ref = notas_usuario(1).index
recomendacoes = notas_similar.drop(filmes_ref, errors='ignore').sort_values('nota', ascending = False)
recomendacoes

,nota
id_filme,
10,5.0
370,5.0
1722,5.0
2011,5.0
7573,5.0
...,...
1391,0.5
3000,0.5
2762,0.5


> ### Generalizando em uma função

In [28]:
def sugestao(id_ref, n_ref):
  proximos(id_ref, n = n_ref)
  similares = proximos(id_ref, n = n_ref)
  similar = similares.iloc[0].name
  notas_similar = notas_usuario(similar)
  filmes_ref = notas_usuario(id_ref).index
  recomendacoes = notas_similar.drop(filmes_ref, errors='ignore').sort_values('nota', ascending = False)
  return recomendacoes

In [29]:
sugestao(1,48)

,nota
id_filme,
10,5.0
370,5.0
1722,5.0
2011,5.0
7573,5.0
...,...
1391,0.5
3000,0.5
2762,0.5


> ## Finalizando Recomendação

Recomendando o filme baseado em mais de um usuário

In [40]:
def proximos(ref_id, n = None, n_proximos = 10):
  distancia = distancia_todos(ref_id, n = n)
  return distancia.sort_values('distancia').query('outro_usuario_id != %d' % ref_id).head(n_proximos)

def sugestao(id_ref, n_ref = None):

  notas_ref = notas_usuario(id_ref)
  filmes_ref = notas_usuario(id_ref).index

  similares = proximos(id_ref, n = n_ref).index
  notas_similares = notas.set_index(['id_usuario']).loc[similares]


  recomendacoes = notas_similares.groupby('id_filme').mean()[['nota']]
  recomendacoes = recomendacoes.sort_values('nota', ascending = False)
  return recomendacoes.join(filmes)

sugestao(1, 500)

,nota,titulo,genero,total_votos,media_nota
id_filme,,,,,
2078,5.0,"Jungle Book, The (1967)",Animation|Children|Comedy|Musical,53.0,3.830189
1792,5.0,U.S. Marshals (1998),Action|Crime|Thriller,23.0,3.021739
3798,5.0,What Lies Beneath (2000),Drama|Horror|Mystery,25.0,3.200000
52328,5.0,Sunshine (2007),Adventure|Drama|Sci-Fi|Thriller,14.0,3.642857
4901,5.0,Spy Game (2001),Action|Crime|Drama|Thriller,27.0,3.481481
...,...,...,...,...,...
107013,0.5,"Christmas Carol, A (1977)",Drama|Fantasy,1.0,0.500000
786,0.5,Eraser (1996),Action|Drama|Thriller,64.0,3.109375
509,0.5,"Piano, The (1993)",Drama|Romance,61.0,3.540984
